# Reference: [구글 BERT의 정석](http://www.yes24.com/Product/Goods/104491152)

## <span style='color:gold'>01. 트랜스포머의 등장 이유</span>

- 트랜스포머 이전에 RNN, LSTM 네트워크가 존재했다(다음 단어 예측/next word prediction, 기계번역/machine translation, 텍스트 생성/text generation 등의 순차적 태스크에서 널리 사용되었음).
- 하지만 이러한 네트워크는 <span style='color:red'>장기 의존성 문제/long-term dependency가 있다.</span>
>**[ long-term dependency ]<br>**
RNN이 hidden state를 통해 과거의 정보를 저장할 때 문장의 길이가 길어지면 앞의 과거 정보가 마지막 시점까지 전달되지 못하는 현상.<br>
-> 모델 학습에 있어 과거의 정보를 활용하지 못하기에 긴 문장이 입력됐을 때 성능이 떨어진다.
- 순수하게 어텐션/attention만을 사용한 모델인 트랜스포머는 RNN, LSTM의 한계점을 극복하기 위해 등장했다. (원 논문: [Attention is All You Need](https://arxiv.org/abs/1706.03762))

## <span style='color:gold'>02. 인코더</span>

- 트랜스포머의 인코더는 크게 두 가지로 구성되어있다.
> **멀티 헤드 어텐션(Multi-Head Attention, MHA)**<br>
**피드포워드 네트워크(Feedforward Network)**
- MHA를 이해하기 위해선 셀프 어텐션(self attention)을 이해하고 넘어가야 한다.

### 02-01. 셀프 어텐션

**A dog ate the food because it was hugnry.**<br>
- 다음과 같은 문장이 있을 때, it은 'food'가 아닌 'dog'를 의미한다.
- 사람은 문장 해석을 통해 it이 'dog'를 가리킨다는 것을 알 수 있지만, 모델은 어떻게 알 수 있을까? -> 이 때 self attention을 사용한다.
- 모델은 각 단어의 표현을 순차적으로 학습하는데(e.g. A -> dog -> ate -> ...), self attention을 통해 <span style='color:red'>학습하는 과정에서 문장을 구성하는 다른 모든 단어들과의 관계 또한 배우게 된다.</span>
- 간단하게 정리하자면 self attention은 **단어와 단어 사이의 관계를 파악**하기 위해 사용하는 것이라고 할 수 있다.

**self attention의 동작 원리**
- self attention을 계산하기 위해선 Q(Query), K(Key), V(Value) matrix가 필요하다.
- Q, K, V matrix는 입력 matrix로 부터 생성된다.
- 입력 matrix는 (문장 길이 x 임베딩 차원)의 shape을 갖는 matrix이다.

In [2]:
import torch
import torch.nn as nn

**[입력 matrix 생성]**

In [3]:
input_string = 'I am good'
word_list = input_string.split()
print(word_list)

embedding_dim = 512
input_matrix = torch.randn(len(word_list), embedding_dim) # (단어 길이, 임베딩 차원)의 모양을 갖는 입력 행렬 생성
print(input_matrix.shape)

['I', 'am', 'good']
torch.Size([3, 512])


**[Q, K, V matrix 생성]**

In [4]:
qkv_dim = 64
weight_Q = nn.Linear(embedding_dim, qkv_dim)
weight_K = nn.Linear(embedding_dim, qkv_dim)
weight_V = nn.Linear(embedding_dim, qkv_dim)

Q = weight_Q(input_matrix)
K = weight_K(input_matrix)
V = weight_V(input_matrix)

print(f'query: {Q.shape}')
print(f'key: {K.shape}')
print(f'value: {V.shape}')

query: torch.Size([3, 64])
key: torch.Size([3, 64])
value: torch.Size([3, 64])


**[self attention 계산]**

In [5]:
# 01. Q와 K.t() 간 내적 연산
out = torch.matmul(Q, K.t())
print(out)

tensor([[ 0.3030,  1.7906,  0.0725],
        [-1.5858,  2.5118,  1.6223],
        [-7.3314,  1.8375, -1.9029]], grad_fn=<MmBackward0>)


- Q와 K.t() 간 내적 연산의 의미는 무엇인가?
- Q, K, V 행렬을 구할 때 달라진 것은 임베딩 차원일 뿐, **문장의 길이는 똑같다.** 즉 Q, K, V의 각 행은 'I', 'am', 'good'을 표현하는 벡터들을 의미한다.
- Q와 K.t()간의 내적 연산은, 하나의 단어와 전체 단어간의 내적을 구하는 것이라고 생각할 수 있다.
- 출력 결과(3x3 matrix)를 보면 이해하기 쉬운데, **<span style='color:red'>(0, 0)은 'I'와 'I'의 내적, (0, 1)은 'I'와 'am'의 내적, (0, 2)는 'I'와 'good'의 내적을 의미한다.</span>**
- 내적은 **두 벡터 간 유사성**을 의미한다. 'I'와 'am' 간의 내적은, 두 단어 간 유사성을 구하는 것이라고 설명할 수 있다.
- 내적을 기하학적으로 생각해보면, 하나의 벡터를 또 다른 벡터에 투영(projection)시키는 것이다. 벡터의 크기(magnitude)는 특정 벡터를 나타낼 수 있는 속성 중 하나이기에, 투영시킨 벡터의 크기는 곧 두 벡터의 유사도라고 생각할 수 있다.
- **결국 out은 단어와 단어 간 연관성을 나타내는 matrix라고 할 수 있다.** (지금은 값이 아무 의미를 나타내지 않지만, 학습 과정을 통해 단어 간 관계를 파악할 수 있게 된다.)

- 궁금한 점: 입력 문장 matrix 간의 내적을 구해도 될 텐데, 굳이 Q, K, V matrix를 구하는 이유는 무엇인가?
- 내 생각: '학습을 통한 최적화를 가능하게 만들기 위함'이라고 생각한다. 입력 문장 matrix는, 최적화가 불가능한 고정값이다. 허나 Q, K, V matrix는 각각의 weight matrix를 이용해서 생성했기 때문에, 학습 과정을 통한 최적화가 가능하다. Q, K, V matrix가 어떤 방식으로 최적화될 지는 생각하기 어렵지만, loss function을 최적화하는 방향으로 학습될 것은 분명하고 이는 계속해서 정교해진다는 것을 의미한다.
- 실제 이유: 아직 안 찾아봄

In [6]:
# 02. 학습 안정성을 위해 out을 qkv_dim의 제곱근으로 나누어준다. (특별한 이유는 없고, hyperparameter이다.)
# 값을 scaling하면 그만큼 계산되는 gradient도 작아지기 때문에, 학습 시 안정성을 갖출 수 있다.
# 미분은 순간적인 변화량이다. -> 값이 큰 경우 변화량 또한 크다. -> 큰 값으로 weight를 업데이트하면 당연히 크게 변한다. -> 학습이 불안정해진다. -> 따라서 scaling이 필요하다.
from math import sqrt

print(out.dtype)
print(sqrt(qkv_dim))
out = out / sqrt(qkv_dim)
print(out)

torch.float32
8.0
tensor([[ 0.0379,  0.2238,  0.0091],
        [-0.1982,  0.3140,  0.2028],
        [-0.9164,  0.2297, -0.2379]], grad_fn=<DivBackward0>)


In [7]:
# 03. Softmax() 함수를 이용해서 값을 정규화(0~1 사이의 값으로 만드는 것)한다.
softmax = nn.Softmax(dim=1)
attention_score = softmax(out) # 단어 간 연관성을 나타내는 attention score matrix 연산
print(attention_score)

print(torch.sum(attention_score[0, :])) # 행 방향(dim=1) 총합이 1인지 확인
print(torch.sum(attention_score[:, 0])) # 열 방향(dim=0) 총합이 0이 아닌지 확인

tensor([[0.3149, 0.3792, 0.3059],
        [0.2403, 0.4010, 0.3588],
        [0.1635, 0.5143, 0.3222]], grad_fn=<SoftmaxBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(0.7186, grad_fn=<SumBackward0>)


In [14]:
# 04. attention matrix 연산(주의할 점은, 내적이 아닌 weighted sum!)
# weighted sum이기에 이중 for문 말곤 없는 것 같다.

row, col = attention_score.shape
attention = torch.zeros(V.shape)
for i in range(row):
    for j in range(col):
        attention[i, :] += (attention_score[i][j]*V[i, :]) # score를 하나씩 가져와서, V의 행에 곱한 뒤 attention matrix에 더하기

print(attention.shape)

torch.Size([3, 64])


**[self attention의 의미]**

- V matrix의 각 행은 각 단어를 표현하는 벡터 값이다.
- 거기에 attention score를 곱했다. Attention score는 단어 간 연관성을 나타내는 matrix이다.
- 단어 간 연관성과 단어를 곱했기 때문에, 연관성이 높은 단어일 수록 attention matrix에 더 많은 값이 반영될 것이다.
- 즉 'I'를 나타내는 attention에는 'I'의 V값이 많이 반영될 것이고, 'am'을 나타내는 attention에는 'am'의 V 값이 많이 반영될 것이다.
- 쉽게 설명하자면 **함유량이 다르다고 생각할 수 있다.**
- 물리적인 의미를 생각해봤을 때 attention은 **<span style='color:red'>문장의 문맥 정보를 담고 있다고 해석할 수 있겠다.</span>**

### 02-02 멀티 헤드 어텐션(Multi-Head Attention, MHA)

## Transformer 최종 구현

In [9]:
class encoder(nn.Module):
    def __init__():
        super().__init__()
    
    def forward(self, x):
    
class decoder(nn.Module):
    def __init__():
        super().__init__()
        
    def forward(self, x):

IndentationError: ignored

In [ ]:
class Transformer(nn.Module):
    def __init__():
        super().__init__()
        self.encoder = 
        self.decoder = 
        
    def forward(self, x):
        